<a href="https://colab.research.google.com/github/rmnchopra91/gen-ai/blob/main/poc/GELIE_advanced_text_to_sql_rmn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Quetion List:
**1. Does XYZ Limited Fund have an LIE issued?(Data available)**

Example1: Does CITIBANK CREDIT CARD ISSUANCE TRUST Limited Fund have an LIE issued?
Answer: Yes, it exists. Its LEI is R8AG6FXVDQAMDKCSJJ64
Example2: Does HDFC TOP 200 Limited Fund have an LIE issued?
Answer: It does not exist

**2. Who is managing the XYZ fund? What is Citigroup Inc.'s registered address?**

  Solution: df_citi[df_citi["Relationship.RelationshipType"] == "IS_FUND-MANAGED_BY"]
  
  What is Citigroup Inc.'s registered address?
  Solution: directly available

**3. Where is Citigroup Inc registered?**

eg query: Select * from  table name where legal entity name is "citi inc"
                response: LegalAddress, HeadquartersAddress

**4. Who is the ultimate parent of XYZ Inc?**

                Solution: df2[df2[start_node] == lei_id]

**5. Who is the immediate parent of ABC Inc?**

df2 I ["Relationship. EndNode-NodeID", "Relationship.RelationshipType"]] [df2 [start_nodel = lei_idl

**6. Is XYZ Limited active?**


In [ ]:
! pip install llama-index-llms-openai
! pip install llama-index-vector-stores-pinecone
! pip install llama-index-readers-wikipedia
! pip install llama-index-embeddings-openai
! pip install arize-phoenix
! pip install llama-index-callbacks-arize-phoenix
! pip install llama-index-callbacks-arize-phoenix

# Visualize query pipeline
! pip install pyvis

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
import os

base_path = "/content/drive/MyDrive/Gen Ai/POC"

secret_base_path = "/content/drive/MyDrive/Gen Ai/secrets"
openai_api_key_path = f"{secret_base_path}/openAI_key.txt"

with open(openai_api_key_path, "r") as f:
  os.environ["OPENAI_API_KEY"] = ' '.join(f.readlines())

In [4]:
# This file contains 27 Lakh records and each row having 337 columns
lei_file_path = f"{base_path}/data/filtered_lei_data.csv"
relationship_file_path = f"{base_path}/data/relationship-golden.csv"

In [5]:
import pandas as pd
df = pd.read_csv(lei_file_path)
df_relationship = pd.read_csv(relationship_file_path)

In [6]:
print(f"df shape : {df.shape}")
print(f"df_relationship shape : {df_relationship.shape}")

df shape : (1000, 338)
df_relationship shape : (798, 54)


In [7]:
df.head()

,Unnamed: 0,LEI,Entity.LegalName,Entity.LegalName.xmllang,Entity.OtherEntityNames.OtherEntityName.1,Entity.OtherEntityNames.OtherEntityName.1.xmllang,Entity.OtherEntityNames.OtherEntityName.1.type,Entity.OtherEntityNames.OtherEntityName.2,Entity.OtherEntityNames.OtherEntityName.2.xmllang,Entity.OtherEntityNames.OtherEntityName.2.type,...,Registration.OtherValidationAuthorities.OtherValidationAuthority.2.ValidationAuthorityEntityID,Registration.OtherValidationAuthorities.OtherValidationAuthority.3.ValidationAuthorityID,Registration.OtherValidationAuthorities.OtherValidationAuthority.3.OtherValidationAuthorityID,Registration.OtherValidationAuthorities.OtherValidationAuthority.3.ValidationAuthorityEntityID,Registration.OtherValidationAuthorities.OtherValidationAuthority.4.ValidationAuthorityID,Registration.OtherValidationAuthorities.OtherValidationAuthority.4.OtherValidationAuthorityID,Registration.OtherValidationAuthorities.OtherValidationAuthority.4.ValidationAuthorityEntityID,Registration.OtherValidationAuthorities.OtherValidationAuthority.5.ValidationAuthorityID,Registration.OtherValidationAuthorities.OtherValidationAuthority.5.OtherValidationAuthorityID,Registration.OtherValidationAuthorities.OtherValidationAuthority.5.ValidationAuthorityEntityID
0,1509000,699400ZVAFUI260E7P22,La Renaissance,fr,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1509001,699400ZVBGDG9OZG6J49,TEAL PARTNERS,nl,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1509002,699400ZVTMK0MBXFK186,PENSION ARCHITECTS,nl,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1509003,699400ZWR9DLSZLJLJ75,VOKA-KAMER VAN KOOPHANDEL OOST-VLAANDEREN,nl,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1509004,699400ZWTNE0PT7GPZ58,VIGILANT FINANCIAL ADVISORY,fr,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df_relationship.head()

,Relationship.StartNode.NodeID,Relationship.StartNode.NodeIDType,Relationship.EndNode.NodeID,Relationship.EndNode.NodeIDType,Relationship.RelationshipType,Relationship.RelationshipStatus,Relationship.Period.1.startDate,Relationship.Period.1.endDate,Relationship.Period.1.periodType,Relationship.Period.2.startDate,...,Relationship.Quantifiers.5.QuantifierAmount,Relationship.Quantifiers.5.QuantifierUnits,Registration.InitialRegistrationDate,Registration.LastUpdateDate,Registration.RegistrationStatus,Registration.NextRenewalDate,Registration.ManagingLOU,Registration.ValidationSources,Registration.ValidationDocuments,Registration.ValidationReference
0,029200343C4N7D2E6H83,LEI,6SHGI4ZSSLCXXQSBB395,LEI,IS_ULTIMATELY_CONSOLIDATED_BY,ACTIVE,2021-01-01T00:00:00+00:00,2021-12-31T00:00:00+00:00,ACCOUNTING_PERIOD,2017-07-12T00:00:00+00:00,...,NaN,NaN,2015-02-20T00:00:00+00:00,2024-04-21T10:25:13+00:00,PUBLISHED,2025-05-27T14:05:45+00:00,529900T8BM49AURSDO55,ENTITY_SUPPLIED_ONLY,ACCOUNTS_FILING,https://www.sec.gov/ix?doc=/Archives/edgar/dat...
1,073LKXEKEP6T62NK2G42,LEI,6SHGI4ZSSLCXXQSBB395,LEI,IS_ULTIMATELY_CONSOLIDATED_BY,ACTIVE,2021-01-01T00:00:00+00:00,2021-12-31T00:00:00+00:00,ACCOUNTING_PERIOD,2017-10-02T00:00:00+00:00,...,NaN,NaN,2012-11-28T00:00:00+00:00,2024-04-22T18:24:42+00:00,PUBLISHED,2025-05-27T14:05:43+00:00,529900T8BM49AURSDO55,ENTITY_SUPPLIED_ONLY,SUPPORTING_DOCUMENTS,https://www.ffiec.gov/npw/Institution/Profile/...
2,0C45RG70GW42XSBLJ131,LEI,6SHGI4ZSSLCXXQSBB395,LEI,IS_ULTIMATELY_CONSOLIDATED_BY,ACTIVE,2024-04-27T00:00:00+00:00,NaN,DOCUMENT_FILING_PERIOD,2022-01-01T00:00:00+00:00,...,NaN,NaN,2012-06-27T00:00:00+00:00,2024-04-27T14:18:23+00:00,PUBLISHED,2025-05-27T14:17:44+00:00,529900T8BM49AURSDO55,FULLY_CORROBORATED,REGULATORY_FILING,https://www.ffiec.gov/npw/Institution/Profile/...
3,18B0XXQZGRUMTWTLAN28,LEI,6SHGI4ZSSLCXXQSBB395,LEI,IS_ULTIMATELY_CONSOLIDATED_BY,ACTIVE,2021-01-01T00:00:00+00:00,2021-12-31T00:00:00+00:00,ACCOUNTING_PERIOD,2017-10-02T00:00:00+00:00,...,NaN,NaN,2012-06-06T00:00:00+00:00,2023-05-02T13:40:34+00:00,PUBLISHED,2024-05-27T14:18:14+00:00,529900T8BM49AURSDO55,ENTITY_SUPPLIED_ONLY,SUPPORTING_DOCUMENTS,NaN
4,1P0UUG7LC1S92QTNHG39,LEI,6SHGI4ZSSLCXXQSBB395,LEI,IS_ULTIMATELY_CONSOLIDATED_BY,ACTIVE,2021-01-01T00:00:00+00:00,2021-12-31T00:00:00+00:00,ACCOUNTING_PERIOD,2017-07-12T00:00:00+00:00,...,NaN,NaN,2012-06-06T00:00:00+00:00,2024-04-27T15:10:27+00:00,PUBLISHED,2025-05-27T14:17:56+00:00,529900T8BM49AURSDO55,FULLY_CORROBORATED,SUPPORTING_DOCUMENTS,https://www.sec.gov/ix?doc=/Archives/edgar/dat...


In [9]:

df_entity = df[["LEI", "Entity.LegalName", "Entity.LegalAddress.FirstAddressLine", "Entity.LegalAddress.AddressNumber",
                "Entity.LegalAddress.AddressNumberWithinBuilding", "Entity.LegalAddress.MailRouting",
                "Entity.LegalAddress.AdditionalAddressLine.1", "Entity.LegalAddress.AdditionalAddressLine.2",
                "Entity.LegalAddress.AdditionalAddressLine.3", "Entity.LegalAddress.City", "Entity.LegalAddress.Region",
                "Entity.LegalAddress.Country", "Entity.LegalAddress.PostalCode",
                "Entity.HeadquartersAddress.xmllang", "Entity.HeadquartersAddress.FirstAddressLine",
                "Entity.HeadquartersAddress.AddressNumber", "Entity.RegistrationAuthority.RegistrationAuthorityID",
                "Entity.RegistrationAuthority.RegistrationAuthorityEntityID", "Entity.LegalJurisdiction",
                "Entity.EntityCategory", "Entity.EntitySubCategory", "Entity.EntityStatus",
                "Entity.EntityCreationDate", "Entity.EntityExpirationDate", "Entity.LegalEntityEvents.LegalEntityEvent.1.group_type", "Entity.LegalEntityEvents.LegalEntityEvent.1.event_status",
                "Registration.InitialRegistrationDate", "Registration.LastUpdateDate", "Registration.RegistrationStatus",
                "Registration.NextRenewalDate", "Registration.ManagingLOU", "Registration.ValidationSources",
                "Registration.ValidationAuthority.ValidationAuthorityID"]]

In [10]:
print(f"df_entity shape : {df_entity.shape}")
df_entity.head()

df_entity shape : (1000, 33)


,LEI,Entity.LegalName,Entity.LegalAddress.FirstAddressLine,Entity.LegalAddress.AddressNumber,Entity.LegalAddress.AddressNumberWithinBuilding,Entity.LegalAddress.MailRouting,Entity.LegalAddress.AdditionalAddressLine.1,Entity.LegalAddress.AdditionalAddressLine.2,Entity.LegalAddress.AdditionalAddressLine.3,Entity.LegalAddress.City,...,Entity.EntityExpirationDate,Entity.LegalEntityEvents.LegalEntityEvent.1.group_type,Entity.LegalEntityEvents.LegalEntityEvent.1.event_status,Registration.InitialRegistrationDate,Registration.LastUpdateDate,Registration.RegistrationStatus,Registration.NextRenewalDate,Registration.ManagingLOU,Registration.ValidationSources,Registration.ValidationAuthority.ValidationAuthorityID
0,699400ZVAFUI260E7P22,La Renaissance,Boulevard de la Meuse(JB) 33,33.0,NaN,NaN,NaN,NaN,NaN,Namur,...,NaN,NaN,NaN,2024-02-17T07:02:28+01:00,2024-02-17T07:02:28+01:00,ISSUED,2025-02-17T07:02:28+01:00,894500IIP432AHQ64V02,FULLY_CORROBORATED,RA000025
1,699400ZVBGDG9OZG6J49,TEAL PARTNERS,Damplein 23,23.0,NaN,NaN,NaN,NaN,NaN,Antwerpen,...,NaN,STANDALONE,COMPLETED,2020-10-14T16:23:21+02:00,2023-09-08T07:02:23+02:00,ISSUED,2024-10-14T16:23:21+02:00,894500IIP432AHQ64V02,FULLY_CORROBORATED,RA000025
2,699400ZVTMK0MBXFK186,PENSION ARCHITECTS,Rubenslaan 20,20.0,NaN,NaN,NaN,NaN,NaN,Zoersel,...,NaN,STANDALONE,COMPLETED,2021-12-09T11:24:25+01:00,2023-11-13T18:35:54+01:00,ISSUED,2024-12-09T11:24:25+01:00,894500IIP432AHQ64V02,FULLY_CORROBORATED,RA000025
3,699400ZWR9DLSZLJLJ75,VOKA-KAMER VAN KOOPHANDEL OOST-VLAANDEREN,Lammerstraat 18,18.0,NaN,NaN,NaN,NaN,NaN,Gent,...,NaN,NaN,NaN,2022-05-11T16:06:26+02:00,2023-04-14T07:03:00+02:00,ISSUED,2024-05-11T16:06:26+02:00,894500IIP432AHQ64V02,FULLY_CORROBORATED,RA000025
4,699400ZWTNE0PT7GPZ58,VIGILANT FINANCIAL ADVISORY,Place de la Gare 4,4.0,NaN,NaN,NaN,NaN,NaN,Woluwe-Saint-Pierre,...,NaN,STANDALONE,COMPLETED,2023-01-28T07:02:05+01:00,2023-12-13T07:03:48+01:00,ISSUED,2025-01-28T07:02:05+01:00,894500IIP432AHQ64V02,FULLY_CORROBORATED,RA000025


In [11]:
df_entity[df_entity["LEI"] == "6SHGI4ZSSLCXXQSBB395"]

,LEI,Entity.LegalName,Entity.LegalAddress.FirstAddressLine,Entity.LegalAddress.AddressNumber,Entity.LegalAddress.AddressNumberWithinBuilding,Entity.LegalAddress.MailRouting,Entity.LegalAddress.AdditionalAddressLine.1,Entity.LegalAddress.AdditionalAddressLine.2,Entity.LegalAddress.AdditionalAddressLine.3,Entity.LegalAddress.City,...,Entity.EntityExpirationDate,Entity.LegalEntityEvents.LegalEntityEvent.1.group_type,Entity.LegalEntityEvents.LegalEntityEvent.1.event_status,Registration.InitialRegistrationDate,Registration.LastUpdateDate,Registration.RegistrationStatus,Registration.NextRenewalDate,Registration.ManagingLOU,Registration.ValidationSources,Registration.ValidationAuthority.ValidationAuthorityID
283,6SHGI4ZSSLCXXQSBB395,CITIGROUP INC.,CORPORATION TRUST CENTER 1209 ORANGE ST,NaN,NaN,c/o THE CORPORATION TRUST COMPANY,NaN,NaN,NaN,WILMINGTON,...,NaN,NaN,NaN,2012-06-06T15:53:00+00:00,2023-04-03T10:17:01+00:00,ISSUED,2024-05-27T14:10:35+00:00,529900T8BM49AURSDO55,FULLY_CORROBORATED,RA000602


In [12]:
df_citi = df_entity[df_entity["LEI"] == "6SHGI4ZSSLCXXQSBB395"]

In [13]:
df_citi.head()

,LEI,Entity.LegalName,Entity.LegalAddress.FirstAddressLine,Entity.LegalAddress.AddressNumber,Entity.LegalAddress.AddressNumberWithinBuilding,Entity.LegalAddress.MailRouting,Entity.LegalAddress.AdditionalAddressLine.1,Entity.LegalAddress.AdditionalAddressLine.2,Entity.LegalAddress.AdditionalAddressLine.3,Entity.LegalAddress.City,...,Entity.EntityExpirationDate,Entity.LegalEntityEvents.LegalEntityEvent.1.group_type,Entity.LegalEntityEvents.LegalEntityEvent.1.event_status,Registration.InitialRegistrationDate,Registration.LastUpdateDate,Registration.RegistrationStatus,Registration.NextRenewalDate,Registration.ManagingLOU,Registration.ValidationSources,Registration.ValidationAuthority.ValidationAuthorityID
283,6SHGI4ZSSLCXXQSBB395,CITIGROUP INC.,CORPORATION TRUST CENTER 1209 ORANGE ST,NaN,NaN,c/o THE CORPORATION TRUST COMPANY,NaN,NaN,NaN,WILMINGTON,...,NaN,NaN,NaN,2012-06-06T15:53:00+00:00,2023-04-03T10:17:01+00:00,ISSUED,2024-05-27T14:10:35+00:00,529900T8BM49AURSDO55,FULLY_CORROBORATED,RA000602


In [14]:
df_short = pd.concat([df_citi, df_entity[:5]])

In [15]:
print(f"df_short shape : {df_short.shape}")
df_short.head()

df_short shape : (6, 33)


,LEI,Entity.LegalName,Entity.LegalAddress.FirstAddressLine,Entity.LegalAddress.AddressNumber,Entity.LegalAddress.AddressNumberWithinBuilding,Entity.LegalAddress.MailRouting,Entity.LegalAddress.AdditionalAddressLine.1,Entity.LegalAddress.AdditionalAddressLine.2,Entity.LegalAddress.AdditionalAddressLine.3,Entity.LegalAddress.City,...,Entity.EntityExpirationDate,Entity.LegalEntityEvents.LegalEntityEvent.1.group_type,Entity.LegalEntityEvents.LegalEntityEvent.1.event_status,Registration.InitialRegistrationDate,Registration.LastUpdateDate,Registration.RegistrationStatus,Registration.NextRenewalDate,Registration.ManagingLOU,Registration.ValidationSources,Registration.ValidationAuthority.ValidationAuthorityID
283,6SHGI4ZSSLCXXQSBB395,CITIGROUP INC.,CORPORATION TRUST CENTER 1209 ORANGE ST,NaN,NaN,c/o THE CORPORATION TRUST COMPANY,NaN,NaN,NaN,WILMINGTON,...,NaN,NaN,NaN,2012-06-06T15:53:00+00:00,2023-04-03T10:17:01+00:00,ISSUED,2024-05-27T14:10:35+00:00,529900T8BM49AURSDO55,FULLY_CORROBORATED,RA000602
0,699400ZVAFUI260E7P22,La Renaissance,Boulevard de la Meuse(JB) 33,33.0,NaN,NaN,NaN,NaN,NaN,Namur,...,NaN,NaN,NaN,2024-02-17T07:02:28+01:00,2024-02-17T07:02:28+01:00,ISSUED,2025-02-17T07:02:28+01:00,894500IIP432AHQ64V02,FULLY_CORROBORATED,RA000025
1,699400ZVBGDG9OZG6J49,TEAL PARTNERS,Damplein 23,23.0,NaN,NaN,NaN,NaN,NaN,Antwerpen,...,NaN,STANDALONE,COMPLETED,2020-10-14T16:23:21+02:00,2023-09-08T07:02:23+02:00,ISSUED,2024-10-14T16:23:21+02:00,894500IIP432AHQ64V02,FULLY_CORROBORATED,RA000025
2,699400ZVTMK0MBXFK186,PENSION ARCHITECTS,Rubenslaan 20,20.0,NaN,NaN,NaN,NaN,NaN,Zoersel,...,NaN,STANDALONE,COMPLETED,2021-12-09T11:24:25+01:00,2023-11-13T18:35:54+01:00,ISSUED,2024-12-09T11:24:25+01:00,894500IIP432AHQ64V02,FULLY_CORROBORATED,RA000025
3,699400ZWR9DLSZLJLJ75,VOKA-KAMER VAN KOOPHANDEL OOST-VLAANDEREN,Lammerstraat 18,18.0,NaN,NaN,NaN,NaN,NaN,Gent,...,NaN,NaN,NaN,2022-05-11T16:06:26+02:00,2023-04-14T07:03:00+02:00,ISSUED,2024-05-11T16:06:26+02:00,894500IIP432AHQ64V02,FULLY_CORROBORATED,RA000025


### rename column name

In [16]:
def synthesize_col_name(df):
  columns = df.columns
  updated_columns = []
  for col in columns:
    new_col = col.replace(".", "_")
    updated_columns.append(new_col)
  df.columns = updated_columns
  return df

In [17]:
df_short = synthesize_col_name(df_short)
df_relationship = synthesize_col_name(df_relationship)

In [18]:
df_short

,LEI,Entity_LegalName,Entity_LegalAddress_FirstAddressLine,Entity_LegalAddress_AddressNumber,Entity_LegalAddress_AddressNumberWithinBuilding,Entity_LegalAddress_MailRouting,Entity_LegalAddress_AdditionalAddressLine_1,Entity_LegalAddress_AdditionalAddressLine_2,Entity_LegalAddress_AdditionalAddressLine_3,Entity_LegalAddress_City,...,Entity_EntityExpirationDate,Entity_LegalEntityEvents_LegalEntityEvent_1_group_type,Entity_LegalEntityEvents_LegalEntityEvent_1_event_status,Registration_InitialRegistrationDate,Registration_LastUpdateDate,Registration_RegistrationStatus,Registration_NextRenewalDate,Registration_ManagingLOU,Registration_ValidationSources,Registration_ValidationAuthority_ValidationAuthorityID
283,6SHGI4ZSSLCXXQSBB395,CITIGROUP INC.,CORPORATION TRUST CENTER 1209 ORANGE ST,NaN,NaN,c/o THE CORPORATION TRUST COMPANY,NaN,NaN,NaN,WILMINGTON,...,NaN,NaN,NaN,2012-06-06T15:53:00+00:00,2023-04-03T10:17:01+00:00,ISSUED,2024-05-27T14:10:35+00:00,529900T8BM49AURSDO55,FULLY_CORROBORATED,RA000602
0,699400ZVAFUI260E7P22,La Renaissance,Boulevard de la Meuse(JB) 33,33.0,NaN,NaN,NaN,NaN,NaN,Namur,...,NaN,NaN,NaN,2024-02-17T07:02:28+01:00,2024-02-17T07:02:28+01:00,ISSUED,2025-02-17T07:02:28+01:00,894500IIP432AHQ64V02,FULLY_CORROBORATED,RA000025
1,699400ZVBGDG9OZG6J49,TEAL PARTNERS,Damplein 23,23.0,NaN,NaN,NaN,NaN,NaN,Antwerpen,...,NaN,STANDALONE,COMPLETED,2020-10-14T16:23:21+02:00,2023-09-08T07:02:23+02:00,ISSUED,2024-10-14T16:23:21+02:00,894500IIP432AHQ64V02,FULLY_CORROBORATED,RA000025
2,699400ZVTMK0MBXFK186,PENSION ARCHITECTS,Rubenslaan 20,20.0,NaN,NaN,NaN,NaN,NaN,Zoersel,...,NaN,STANDALONE,COMPLETED,2021-12-09T11:24:25+01:00,2023-11-13T18:35:54+01:00,ISSUED,2024-12-09T11:24:25+01:00,894500IIP432AHQ64V02,FULLY_CORROBORATED,RA000025
3,699400ZWR9DLSZLJLJ75,VOKA-KAMER VAN KOOPHANDEL OOST-VLAANDEREN,Lammerstraat 18,18.0,NaN,NaN,NaN,NaN,NaN,Gent,...,NaN,NaN,NaN,2022-05-11T16:06:26+02:00,2023-04-14T07:03:00+02:00,ISSUED,2024-05-11T16:06:26+02:00,894500IIP432AHQ64V02,FULLY_CORROBORATED,RA000025
4,699400ZWTNE0PT7GPZ58,VIGILANT FINANCIAL ADVISORY,Place de la Gare 4,4.0,NaN,NaN,NaN,NaN,NaN,Woluwe-Saint-Pierre,...,NaN,STANDALONE,COMPLETED,2023-01-28T07:02:05+01:00,2023-12-13T07:03:48+01:00,ISSUED,2025-01-28T07:02:05+01:00,894500IIP432AHQ64V02,FULLY_CORROBORATED,RA000025


In [19]:
df_relationship.head()

,Relationship_StartNode_NodeID,Relationship_StartNode_NodeIDType,Relationship_EndNode_NodeID,Relationship_EndNode_NodeIDType,Relationship_RelationshipType,Relationship_RelationshipStatus,Relationship_Period_1_startDate,Relationship_Period_1_endDate,Relationship_Period_1_periodType,Relationship_Period_2_startDate,...,Relationship_Quantifiers_5_QuantifierAmount,Relationship_Quantifiers_5_QuantifierUnits,Registration_InitialRegistrationDate,Registration_LastUpdateDate,Registration_RegistrationStatus,Registration_NextRenewalDate,Registration_ManagingLOU,Registration_ValidationSources,Registration_ValidationDocuments,Registration_ValidationReference
0,029200343C4N7D2E6H83,LEI,6SHGI4ZSSLCXXQSBB395,LEI,IS_ULTIMATELY_CONSOLIDATED_BY,ACTIVE,2021-01-01T00:00:00+00:00,2021-12-31T00:00:00+00:00,ACCOUNTING_PERIOD,2017-07-12T00:00:00+00:00,...,NaN,NaN,2015-02-20T00:00:00+00:00,2024-04-21T10:25:13+00:00,PUBLISHED,2025-05-27T14:05:45+00:00,529900T8BM49AURSDO55,ENTITY_SUPPLIED_ONLY,ACCOUNTS_FILING,https://www.sec.gov/ix?doc=/Archives/edgar/dat...
1,073LKXEKEP6T62NK2G42,LEI,6SHGI4ZSSLCXXQSBB395,LEI,IS_ULTIMATELY_CONSOLIDATED_BY,ACTIVE,2021-01-01T00:00:00+00:00,2021-12-31T00:00:00+00:00,ACCOUNTING_PERIOD,2017-10-02T00:00:00+00:00,...,NaN,NaN,2012-11-28T00:00:00+00:00,2024-04-22T18:24:42+00:00,PUBLISHED,2025-05-27T14:05:43+00:00,529900T8BM49AURSDO55,ENTITY_SUPPLIED_ONLY,SUPPORTING_DOCUMENTS,https://www.ffiec.gov/npw/Institution/Profile/...
2,0C45RG70GW42XSBLJ131,LEI,6SHGI4ZSSLCXXQSBB395,LEI,IS_ULTIMATELY_CONSOLIDATED_BY,ACTIVE,2024-04-27T00:00:00+00:00,NaN,DOCUMENT_FILING_PERIOD,2022-01-01T00:00:00+00:00,...,NaN,NaN,2012-06-27T00:00:00+00:00,2024-04-27T14:18:23+00:00,PUBLISHED,2025-05-27T14:17:44+00:00,529900T8BM49AURSDO55,FULLY_CORROBORATED,REGULATORY_FILING,https://www.ffiec.gov/npw/Institution/Profile/...
3,18B0XXQZGRUMTWTLAN28,LEI,6SHGI4ZSSLCXXQSBB395,LEI,IS_ULTIMATELY_CONSOLIDATED_BY,ACTIVE,2021-01-01T00:00:00+00:00,2021-12-31T00:00:00+00:00,ACCOUNTING_PERIOD,2017-10-02T00:00:00+00:00,...,NaN,NaN,2012-06-06T00:00:00+00:00,2023-05-02T13:40:34+00:00,PUBLISHED,2024-05-27T14:18:14+00:00,529900T8BM49AURSDO55,ENTITY_SUPPLIED_ONLY,SUPPORTING_DOCUMENTS,NaN
4,1P0UUG7LC1S92QTNHG39,LEI,6SHGI4ZSSLCXXQSBB395,LEI,IS_ULTIMATELY_CONSOLIDATED_BY,ACTIVE,2021-01-01T00:00:00+00:00,2021-12-31T00:00:00+00:00,ACCOUNTING_PERIOD,2017-07-12T00:00:00+00:00,...,NaN,NaN,2012-06-06T00:00:00+00:00,2024-04-27T15:10:27+00:00,PUBLISHED,2025-05-27T14:17:56+00:00,529900T8BM49AURSDO55,FULLY_CORROBORATED,SUPPORTING_DOCUMENTS,https://www.sec.gov/ix?doc=/Archives/edgar/dat...


### list of dataframes

In [20]:
dfs = [df_short, df_relationship]

### Extract Table Name and Summary from each Table

In [21]:
tableinfo_dir = f"{base_path}/data/tables"
# !mkdir {tableinfo_dir}

In [22]:
from llama_index.core.bridge.pydantic import BaseModel, Field

class TableInfo(BaseModel):
    """Information regarding a structured table."""

    table_name: str = Field(
        ..., description="table name (must be underscores and NO spaces)"
    )
    table_summary: str = Field(
        ..., description="short, concise summary/caption of the table"
    )

In [23]:
prompt_str = """\
Give me a summary of the table with the following json format.

- Table name must be unique to the table and describe it while being concise.
- Do not output a Generic table name (e.g. table, my_table)

Do not make table name one of the following: {exclude_table_name_list}

Table: {table_str}
Summary:
"""

In [24]:
from llama_index.core.program import LLMTextCompletionProgram
from llama_index.llms.openai import OpenAI


program = LLMTextCompletionProgram.from_defaults(
    output_cls = TableInfo,
    llm = OpenAI(model="gpt-3.5-turbo"),
    prompt_template_str = prompt_str
)

In [25]:
import json
from pathlib import Path

def _get_tableinfo_with_index(idx: int) -> str:
  results_gen = Path(tableinfo_dir).glob(f"{idx}_*")
  results_list = list(results_gen)
  if len(results_list) == 0:
    return None
  elif len(results_list) == 1:
    path = results_list[0]
    return TableInfo.parse_file(path)
  else:
    raise ValueError(
        f"More than one file matching index: {list(results_gen)}"
    )

In [26]:
table_names = set()
table_infos = []

In [27]:
len(dfs)

2

In [28]:
for idx, df in enumerate(dfs):
  table_info = _get_tableinfo_with_index(idx)
  print("table_info -- ", table_info)
  if table_info:
    table_infos.append(table_info)
  else:
    while True:
      df_str = df.head(10).to_csv()
      table_info = program(
          table_str = df_str,
          exclude_table_name_list = str(list(table_names))
      )
      print("table_info -->> ", table_info)
      table_name = table_info.table_name
      print(f"Processed table: {table_name}")
      if table_name not in table_names:
        table_names.add(table_name)
        break
      else:
        print(f"Table name {table_name} already exist, trying again.")
        pass

    out_file = f"{tableinfo_dir}/{idx}_{table_name}.json"
    json.dump(table_info.dict(), open(out_file, "w"))
    table_infos.append(table_info)

table_info --  None
table_info -->>  table_name='Entity_Information_Table' table_summary='Table containing information about various entities including legal names, addresses, registration details, and entity status.'
Processed table: Entity_Information_Table
table_info --  None
table_info -->>  table_name='Relationship_Information_Table' table_summary='Table containing information about relationships between nodes, including start and end node IDs, relationship types, status, periods, qualifiers, quantifiers, and registration details.'
Processed table: Relationship_Information_Table


In [29]:
len(table_infos)

2

In [30]:
table_infos

[TableInfo(table_name='Entity_Information_Table', table_summary='Table containing information about various entities including legal names, addresses, registration details, and entity status.'),
 TableInfo(table_name='Relationship_Information_Table', table_summary='Table containing information about relationships between nodes, including start and end node IDs, relationship types, status, periods, qualifiers, quantifiers, and registration details.')]

In [31]:
table_infos[1].table_summary = "Empty"